# M2 Bending Mode Test

This Jupyter notebook is to run the bending mode test of M2.
Each axial actuator will be issued with an additioal force according to specific bending mode (there are 20).
The target is to understand the mirror supporting system can hold the specific shape based on bending mode or not.

## Import Modules

In [ ]:
import asyncio
import yaml
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK
from lsst.ts.ofc.utils import get_config_dir as get_config_dir_ofc

## Declaration of User-defined Functions

In [ ]:
def get_bending_mode():
    """Get the bending mode.

    Returns
    -------
    bending_mode : `numpy.ndarray`
        Bending mode.
    """
    bending_mode_file = get_config_dir_ofc() / "M2" / "M2_1um_72_force.yaml"
    with open(bending_mode_file, "r") as yaml_file:
        bending_mode = np.array(yaml.safe_load(yaml_file))

    return bending_mode

In [ ]:
def get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude):
    """Plot the forces of bending mode.
    
    Parameters
    ----------
    bending_mode : `numpy.ndarray`
        Bending mode data.
    idx_bending_mode : `int`
        Index of bending mode (1-20).
    amplitude : `float`
        Amplitude of bending mode in um.
        
    Returns
    -------
    `numpy.ndarray`
        Actuator forces in Newton.
    """

    # Column 0-2 are the actuator ID, x and y position of actuator
    return amplitude * bending_mode[:, 2+idx_bending_mode]

In [ ]:
async def apply_force_cycle_axial(csc, forces, time_forces=5, time_reset=5):
    """Apply the force cycle to axial actuators. The cycle will be positive force, clear,
    negative force, clear.

    Parameters
    ----------
    csc : `lsst.ts.salobj.remote.Remote`
        Remote object of the M2 CSC.
    forces : `numpy.ndarray`
        Axial forces to apply (Newton).
    time_forces : `float`, optional
        Time to apply the forces in second. (the default is 5.0) 
    time_reset : `float`, optional
        Time to reset the forces in second. (the default is 5.0)
    """

    # Do the positive direction first
    print(f"Apply the force: {forces} N.")
    await csc.cmd_applyForces.set_start(axial=forces.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

    # Do the Negative direction
    forces_negative = -forces

    print(f"Apply the force: {forces_negative} N.")
    await csc.cmd_applyForces.set_start(axial=forces_negative.tolist())
    await asyncio.sleep(time_forces)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(time_reset)

## Prepare the M2 CSC and put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=60)

## Apply the Bending Mode

In [ ]:
bending_mode = get_bending_mode()

In [ ]:
idx_bending_mode = 1
amplitude = 1
forces = get_bending_mode_forces(bending_mode, idx_bending_mode, amplitude)

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

In [ ]:
await apply_force_cycle_axial(m2, forces, time_forces=10, time_reset=5)